In [ ]:
import polars as pl

# Ghi ra file Excel
excel_path = r"C:\Users\huuchinh.nguyen\Concentrix Corporation\WFM-Expedia-HCM - Branding files\Performance\Global Report\New Look Excel data_2025_Jun_EN.xlsx"
mapping_file = r"C:\Users\huuchinh.nguyen\Concentrix Corporation\WFM-Expedia-HCM - Branding files\Performance\LOB Mapping Aligned OneReport\AQG_summarized.xlsx"

mapping = pl.read_excel(mapping_file)
df = pl.read_excel(excel_path)

df = df.with_columns(pl.when(pl.col("NPS Raw Score").is_in([7,8]))
      .then(1)
      .otherwise(0)
      .alias("Neutral Score (Calc)"))

df = df.join(mapping, on="Agent Queue Group Name",how='left')
df = df.rename({'CFG Groups':'LOB'})

filtered = mapping.filter(
    pl.col("Agent Queue Group Name").str.contains(r"[dD][aA]")
)

print(filtered)


Could not determine dtype for column 11, falling back to string


shape: (12, 4)
┌────────────────────┬────────────────────────┬────────────────────────────────┬───────────────────┐
│ MainLOBs           ┆ CFG Groups             ┆ Agent Queue Group Name         ┆ Site              │
│ ---                ┆ ---                    ┆ ---                            ┆ ---               │
│ str                ┆ str                    ┆ str                            ┆ str               │
╞════════════════════╪════════════════════════╪════════════════════════════════╪═══════════════════╡
│ Retail English     ┆ Voice Non Lodging      ┆ Voice_GLB_EN_Midas_Outsourced  ┆ Concentrix (Pune) │
│ Retail English     ┆ Voice Non Lodging      ┆ Voice_GLB_EN_Midas_Outsourced_ ┆ Concentrix (Pune) │
│                    ┆                        ┆ …                              ┆                   │
│ Retail English     ┆ Voice Non Lodging      ┆ Voice_GLB_EN_Midas_Outsourced_ ┆ Concentrix (Pune) │
│                    ┆                        ┆ …                           

In [110]:
columns_standard = [
    "Joined Date", "Agent People Id", "Conversation Id", "Agent Email ID", "Agent Queue Group Name",
    "Latest VA Intent", "Latest VA Product", "Business Segment Name", "Partner Name",
    "Handle (Count)", "Handle Time (Sum)", "Response Count", "Response Time", "Hold Time (Sum)", "Wrap Up Time (Sum)",
    "Talk Time (Sum)", "LOB", "Outbound (Count)", "Follow Up (Count)", "Survey Submitted (Count)", "Promoter Score (Calc)", "Detractor Score (Calc)", "Neutral Score (Calc)","MainLOBs","Site"
]

df_columns = df.columns

missing_cols = [col for col in columns_standard if col not in df_columns]

for col in missing_cols:
    df = df.with_columns(pl.lit("").alias(col))
    print(col)

df = df.select(columns_standard)

TEST = df.filter(
    pl.col("Agent Queue Group Name")
      .str.to_lowercase()
      .str.contains("da")
)

# Hiển thị kết quả
print(TEST)

df.write_excel('test.xlsx')
df.head()

shape: (416, 25)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Joined    ┆ Agent     ┆ Conversat ┆ Agent     ┆ … ┆ Detractor ┆ Neutral   ┆ MainLOBs  ┆ Site     │
│ Date      ┆ People Id ┆ ion Id    ┆ Email ID  ┆   ┆ Score     ┆ Score     ┆ ---       ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ (Calc)    ┆ (Calc)    ┆ str       ┆ str      │
│ date      ┆ str       ┆ str       ┆ str       ┆   ┆ ---       ┆ ---       ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ i64       ┆ i32       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2025-06-0 ┆ 436753643 ┆ 2cbdeab8- ┆ michal.ti ┆ … ┆ null      ┆ 0         ┆ Retail    ┆ Concentr │
│ 9         ┆           ┆ 902d-4789 ┆ demann@co ┆   ┆           ┆           ┆ Non       ┆ ix (Hels │
│           ┆           ┆ -9b37-9a3 ┆ ncentrix. ┆   ┆           ┆         

Joined Date,Agent People Id,Conversation Id,Agent Email ID,Agent Queue Group Name,Latest VA Intent,Latest VA Product,Business Segment Name,Partner Name,Handle (Count),Handle Time (Sum),Response Count,Response Time,Hold Time (Sum),Wrap Up Time (Sum),Talk Time (Sum),LOB,Outbound (Count),Follow Up (Count),Survey Submitted (Count),Promoter Score (Calc),Detractor Score (Calc),Neutral Score (Calc),MainLOBs,Site
date,str,str,str,str,str,str,str,str,i64,f64,i64,str,f64,f64,f64,str,i64,i64,i64,i64,i64,i32,str,str
2025-06-09,"""402018""","""d2ac274f-17a3-4bb5-b236-333c39…","""mohamed.moharamfarag@concentri…","""Voice_GLB_ES_Blended""","""VIEW_BOOKING""","""UNKNOWN""","""Expedia United States""","""Expedia""",1,265.678016,0,"""""",0.0,5.721,259.957008,"""Voice Spanish""",0,0,0,null,null,0,"""Retail Non English""","""Concentrix (Cairo)"""
2025-06-09,"""388563""","""c1f34e5c-9506-4d2d-a5bf-8bd5e7…","""ahmed.ahmedhafezmashaal@concen…","""Voice_GLB_ES_Blended""","""CHANGE""","""FLIGHT""","""Expedia Spain""","""Expedia""",1,680.42605,0,"""""",205.289008,68.062,407.075008,"""Voice Spanish""",0,0,0,null,null,0,"""Retail Non English""","""Concentrix (Cairo)"""
2025-06-09,"""368688""","""46422834-f1dc-4f42-8a93-450935…","""hassan.shaabanmahmoudabdelgaye…","""Voice_GLB_ES_Blended""","""CHANGE""","""FLIGHT""","""Expedia United States""","""Expedia""",1,623.11603,0,"""""",0.0,67.979,555.13702,"""Voice Spanish""",0,0,0,null,null,0,"""Retail Non English""","""Concentrix (Cairo)"""
2025-06-09,"""30674287""","""cbb368d1-7d38-4b73-92fd-49ca28…","""khaled.samy@concentrix.com""","""Voice_GLB_DE_All_Product""","""REWARDS""","""FLIGHT""","""Expedia Germany""","""Expedia""",1,658.40506,0,"""""",0.0,1.126,657.27898,"""Voice German""",0,0,0,null,null,0,"""Retail Non English""","""Concentrix (Cairo)"""
2025-06-09,"""6897710""","""42ec3533-dd9c-4364-8612-bed8ee…","""muhammed.elsayedfarag@concentr…","""Voice_GLB_ES_Blended""","""UNKNOWN""","""UNKNOWN""","""Expedia United States""","""Expedia""",1,12.171,0,"""""",0.0,5.522,6.649,"""Voice Spanish""",0,0,0,null,null,0,"""Retail Non English""","""Concentrix (Cairo)"""
